In [1]:
cd ~/tfproject/



SyntaxError: invalid syntax (<ipython-input-1-3fe2971edc07>, line 1)

In [2]:
cd ~/tfproject/

/Users/chengjielin/tfproject


In [3]:
cd Spring-2015-IW-master/


/Users/chengjielin/tfproject/Spring-2015-IW-master


In [4]:
ls -al

total 10536
drwxr-xr-x@ 33 chengjielin  staff     1122 Apr 12  2016 ./
drwxr-xr-x  17 chengjielin  staff      578 Nov 13 00:00 ../
-rwxr-xr-x@  1 chengjielin  staff     8196 Nov 14 16:04 .DS_Store*
-rwxr-xr-x@  1 chengjielin  staff    22734 Apr 12  2016 .Rhistory*
-rwxr-xr-x@  1 chengjielin  staff       19 Apr 12  2016 .gitignore*
drwxr-xr-x@ 10 chengjielin  staff      340 Apr 12  2016 IWformat/
-rwxr-xr-x@  1 chengjielin  staff   206336 Apr 12  2016 IWformat.tar*
drwxr-xr-x@ 20 chengjielin  staff      680 Apr 12  2016 Paper/
-rwxr-xr-x@  1 chengjielin  staff      540 Apr 12  2016 README.md*
-rwxr-xr-x@  1 chengjielin  staff     7568 Apr 12  2016 RealSP500List.prn*
-rwxr-xr-x@  1 chengjielin  staff     9630 Apr 12  2016 RealSP500List.txt*
-rwxr-xr-x@  1 chengjielin  staff    35021 Apr 12  2016 RealSP500List.xlsx*
-rwxr-xr-x@  1 chengjielin  staff   118138 Apr 12  2016 Saahil_Madge.docx*
-rwxr-xr-x@  1 chengjielin  staff     1026 Apr 12  2016 currencyRateAnalysis.py*
-rwxr-xr-x@  1 chen

In [8]:
# %load techsectoranalysis.py
import pandas
import numpy as np
from sklearn import preprocessing
from sklearn import svm
from sklearn import cross_validation

# read the data
df = pandas.read_csv('techsectordatareal.csv')
daysAhead =90

# calculate price volatility array given company
def calcPriceVolatility(numDays, priceArray):
	global daysAhead
	# make price volatility array
	volatilityArray = []
	movingVolatilityArray = []
	for i in range(1, numDays+1):
		percentChange = 100 * (priceArray[i] - priceArray[i-1]) / priceArray[i-1]
		movingVolatilityArray.append(percentChange)
	volatilityArray.append(np.mean(movingVolatilityArray))
	for i in range(numDays + 1, len(priceArray) - daysAhead):
		del movingVolatilityArray[0]
		percentChange = 100 * (priceArray[i] - priceArray[i-1]) / priceArray[i-1]
		movingVolatilityArray.append(percentChange)
		volatilityArray.append(np.mean(movingVolatilityArray))

	return volatilityArray

# calculate momentum array
def calcMomentum(numDays, priceArray):
	global daysAhead
	# now calculate momentum
	momentumArray = []
	movingMomentumArray = []
	for i in range(1, numDays + 1):
		movingMomentumArray.append(1 if priceArray[i] > priceArray[i-1] else -1)
	momentumArray.append(np.mean(movingMomentumArray))
	for i in range(numDays+1, len(priceArray) - daysAhead):
		del movingMomentumArray[0]
		movingMomentumArray.append(1 if priceArray[i] > priceArray[i-1] else -1)
		momentumArray.append(np.mean(movingMomentumArray))

	return momentumArray

def makeModelAndPredict(permno, numDays, sectorVolatility, sectorMomentum, splitNumber):
	global df
	global daysAhead
	# get price volatility and momentum for this company
	companyData = df[df['PERMNO'] == permno]
	companyPrices = list(companyData['PRC'])

	volatilityArray = calcPriceVolatility(numDays, companyPrices)
	momentumArray = calcMomentum(numDays, companyPrices)

	splitIndex = splitNumber - numDays

	# since they are different lengths, find the min length
	if len(volatilityArray) > len(sectorVolatility):
		difference = len(volatilityArray) - len(sectorVolatility)
		del volatilityArray[:difference]
		del momentumArray[:difference]

	elif len(sectorVolatility) > len(volatilityArray):
		difference = len(sectorVolatility) - len(volatilityArray)
		del sectorVolatility[:difference]
		del sectorMomentum[:difference]

	# create the feature vectors X
	X = np.transpose(np.array([volatilityArray, momentumArray, sectorVolatility, sectorMomentum]))

	# create the feature vectors Y
	Y = []
	for i in range(numDays, len(companyPrices) - daysAhead):
		Y.append(1 if companyPrices[i+daysAhead] > companyPrices[i] else -1)
	#print (len(Y))

	# fix the length of Y if necessary
	if len(Y) > len(X):
		#print ("here2")
		difference = len(Y) - len(X)
		del Y[:difference]

	# split into training and testing sets
	X_train = np.array(X[0:splitIndex]).astype('float64')
	X_test = np.array(X[splitIndex:]).astype('float64')
	y_train = np.array(Y[0:splitIndex]).astype('float64')
	y_test = np.array(Y[splitIndex:]).astype('float64')

	# fit the model and calculate its accuracy
	rbf_svm = svm.SVC(kernel='rbf')
	rbf_svm.fit(X_train, y_train)
	score = rbf_svm.score(X_test, y_test)
	#print (score)
	return score

def main():
	global df

	# find the list of companies
	permnoList = sorted(set(list(df['PERMNO'])))
	companiesNotFull = [12084, 13407, 14542, 93002, 15579] # companies without full dates

	# read the tech sector data
	ndxtdf = pandas.read_csv('ndxtdata.csv')
	ndxtdf = ndxtdf.sort_index(by='Date', ascending=True)
	ndxtPrices = list(ndxtdf['Close'])

	# find when 2012 starts
	startOfTwelve = list(df[df['PERMNO'] == 10107]['date']).index(20120103)

	# we want to predict where it will be on the next day based on X days previous
	numDaysArray = [5, 10, 20, 90, 270] # day, week, month, quarter, year

	predictionDict = {}

	# iterate over combinations of n_1 and n_2 and find prediction accuracies
	for numDayIndex in numDaysArray:
		for numDayStock in numDaysArray:
			ndxtVolatilityArray = calcPriceVolatility(numDayIndex, ndxtPrices)
			ndxtMomentumArray = calcMomentum(numDayIndex, ndxtPrices)
			predictionForGivenNumDaysDict = {}

			for permno in permnoList:
				if permno in companiesNotFull:
					continue
				#print (permno)
				percentage = makeModelAndPredict(permno,numDayStock,ndxtVolatilityArray,ndxtMomentumArray,startOfTwelve)
				predictionForGivenNumDaysDict[permno] = percentage


			predictionAccuracies = list(predictionForGivenNumDaysDict.values())
			meanAccuracy = np.mean(predictionAccuracies)
			maxIndex = max(predictionForGivenNumDaysDict, key=predictionForGivenNumDaysDict.get)
			maxAccuracy = (maxIndex, predictionForGivenNumDaysDict[maxIndex])
			minIndex = min(predictionForGivenNumDaysDict, key=predictionForGivenNumDaysDict.get)
			minAccuracy = (minIndex, predictionForGivenNumDaysDict[minIndex])
			median = np.median(predictionAccuracies)

			numDaysTuple = (numDayIndex, numDayStock)
			predictionDict[numDaysTuple] = {'mean':meanAccuracy, 'max':predictionForGivenNumDaysDict[maxIndex], 'min':predictionForGivenNumDaysDict[minIndex], 'median':median }

	sortedTuples = sorted(predictionDict.keys())
	for numDaysTuple in sortedTuples:
		# print "%s:\t %s\n" % (numDaysTuple, predictionDict[numDaysTuple])
		sumStats = predictionDict[numDaysTuple]
		print ("& %d & %d & %f & %f & %f & %f \\\\\n" % (numDaysTuple[0], numDaysTuple[1], sumStats['mean'], sumStats['median'], sumStats['max'], sumStats['min']))

if __name__ == "__main__": 
	main()


/Users/chengjielin/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:106: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


& 5 & 5 & 0.555679 & 0.570030 & 0.849398 & 0.251506 \\

& 5 & 10 & 0.554571 & 0.545934 & 0.849398 & 0.256024 \\

& 5 & 20 & 0.550053 & 0.551205 & 0.849398 & 0.222892 \\

& 5 & 90 & 0.492470 & 0.491717 & 0.849398 & 0.203313 \\

& 5 & 270 & 0.572821 & 0.602410 & 0.814759 & 0.161145 \\

& 10 & 5 & 0.551415 & 0.562974 & 0.848255 & 0.239757 \\

& 10 & 10 & 0.565556 & 0.574548 & 0.849398 & 0.236446 \\

& 10 & 20 & 0.567328 & 0.588102 & 0.849398 & 0.201807 \\

& 10 & 90 & 0.497387 & 0.520331 & 0.849398 & 0.194277 \\

& 10 & 270 & 0.571536 & 0.597139 & 0.774096 & 0.165663 \\

& 20 & 5 & 0.549760 & 0.566256 & 0.852080 & 0.221880 \\

& 20 & 10 & 0.556845 & 0.563456 & 0.847095 & 0.244648 \\

& 20 & 20 & 0.570030 & 0.571536 & 0.849398 & 0.225904 \\

& 20 & 90 & 0.501329 & 0.516566 & 0.849398 & 0.167169 \\

& 20 & 270 & 0.577826 & 0.617470 & 0.796687 & 0.118976 \\

& 90 & 5 & 0.498324 & 0.524180 & 0.911917 & 0.138169 \\

& 90 & 10 & 0.504684 & 0.517123 & 0.912671 & 0.160959 \\

& 90 & 20 & 0.512280